In [ ]:
import boto3
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from io import StringIO
import re

# === Configuración de S3 ===
S3_BUCKET = "headlinesjobs"
RAW_PREFIX = "headlines/raw/"
FINAL_PREFIX = "headlines/final/periodico=eltiempo"
s3_client = boto3.client("s3")

def extract_data_from_html(html_content):
    """Extrae categoría, titular y enlace de artículos de eltiempo.com"""
    soup = BeautifulSoup(html_content, "html.parser")
    news_data = []

    for article in soup.find_all("article", class_="c-articulo--textual"):
        title_tag = article.find("a", class_="c-articulo__titulo__txt")
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)
        link = title_tag.get("href", "Sin enlace")
        category = article.get("data-category", "Sin categoría")
        news_data.append([category, title, link])

    return news_data

def main():
    # Listar todos los archivos HTML bajo headlines/raw/
    response = s3_client.list_objects_v2(Bucket=S3_BUCKET, Prefix=RAW_PREFIX)

    if "Contents" not in response:
        print("❌ No se encontraron archivos en el bucket.")
        return

    for obj in response["Contents"]:
        s3_key = obj["Key"]
        if not s3_key.endswith(".html"):
            continue

        print(f"📄 Procesando archivo: s3://{S3_BUCKET}/{s3_key}")
        html_obj = s3_client.get_object(Bucket=S3_BUCKET, Key=s3_key)
        html_content = html_obj["Body"].read()

        news = extract_data_from_html(html_content)
        if not news:
            print(f"⚠️ No se encontraron noticias en {s3_key}")
            continue

        df = pd.DataFrame(news, columns=["Categoria", "Titular", "Enlace"])

        # Extraer la fecha desde el nombre del archivo
        match = re.search(r"contenido-(\d{4})-(\d{2})-(\d{2})\.html", s3_key)
        if match:
            year, month, day = match.groups()
        else:
            # Si no se puede extraer, usar fecha actual
            now = datetime.utcnow()
            year, month, day = now.strftime("%Y"), now.strftime("%m"), now.strftime("%d")

        output_key = f"{FINAL_PREFIX}/year={year}/month={month}/day={day}/headlines.csv"
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)

        s3_client.put_object(
            Bucket=S3_BUCKET,
            Key=output_key,
            Body=csv_buffer.getvalue()
        )

        print(f"✅ CSV creado en s3://{S3_BUCKET}/{output_key}")

if __name__ == "__main__":
    main()